## Overview

Notebook Utils


## Aim

- https://aimstack.readthedocs.io/en/latest/using/sagemaker_notebook_ui.html
- https://github.com/aimhubio/aim/blob/1eddf0b55e5ae854e99fb93548429e2cfc8fd79c/src/python/aim/_ext/notebook/sagemaker/on-start.sh#L4

In [ ]:
%load_ext aim

In [ ]:
%aim up --proxy-url=https://<instance>.notebook.<region>.sagemaker.aws --port=8086

## Check environment

In [ ]:
!echo $PATH

In [ ]:
!type python

In [ ]:
sys.executable

In [ ]:
!type ls

In [ ]:
!type -a python
!type -a conda
!type -a pip

In [ ]:
import sys
sys.path

In [ ]:
import numpy
numpy.__path__

In [ ]:
paths = !type -a python
for path in set(paths):
    path = path.split()[-1]
    print(path)
    !{path} -c "import sys; print(sys.path)"
    print()

In [ ]:
!conda env list

In [ ]:
!jupyter kernelspec list

In [ ]:
!cat /home/ec2-user/anaconda3/envs/Braket/share/jupyter/kernels/python3/kernel.json

## Install packages

- https://repost.aws/knowledge-center/sagemaker-python-package-conda

In [ ]:
%%bash
curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.3/install.sh | bash
. ~/.nvm/nvm.sh
nvm install 16

In [ ]:
# install the AWS CDK Toolkit globally using npm and check the installation by calling the cdk command to fetch the version.
!python -m pip install aws-cdk-lib

In [ ]:
import os

node_path ="/home/ec2-user/.nvm/versions/node/v16.20.1/bin/" 

os.environ["PATH"] += os.pathsep + node_path

In [ ]:
!sudo npm install -g aws-cdk

In [ ]:
!cdk --version

In [ ]:
%%bash
cd ..
pip install -r requirements.txt

In [ ]:
%%bash
cd ..
cdk synth

In [ ]:
%%bash
cd ..
cdk bootstrap

In [ ]:
%%bash
cd ..
cdk deploy --all --require-approval never

## AWS CLI


In [ ]:
!aws cloudfront list-distributions

## SSH

Amazon SageMaker Notebook Instance + SSH + IDE Integration
- https://github.com/aws-samples/sagemaker-ssh-helper
- https://aws.amazon.com/cn/blogs/china/take-you-ssh-to-amazon-sagemaker-training-example-to-find-out/

### 权限配置

- https://github.com/aws-samples/sagemaker-ssh-helper/blob/main/IAM_SSM_Setup.md
- https://docs.aws.amazon.com/zh_cn/systems-manager/latest/userguide/systems-manager-managedinstances-advanced.html#systems-manager-managedinstances-advanced-enabling

注意配置角色的信任关系

```json
{
    "Effect": "Allow",
    "Principal": {
        "Service": "ssm.amazonaws.com"
    },
    "Action": "sts:AssumeRole"
}
```


In [ ]:
%%sh
pip uninstall -y -q awscli
pip install q -U sagemaker-ssh-helper
pip freeze | grep sagemaker-ssh-helper

In [ ]:
from sagemaker_ssh_helper.wrapper import SSHEstimatorWrapper
from sagemaker.pytorch import PyTorch
import sagemaker
import logging
import os

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
!mkdir -p ssh_notebook/

In [ ]:
%%writefile ssh_notebook/ssh_notebook.py

import time
import os

import sagemaker_ssh_helper
sagemaker_ssh_helper.setup_and_start_ssh()

while os.environ.get("START_SSH", "false") == "true":
    time.sleep(10)  # will sleep forever

In the next cell you need to put your **local** user id. It will help to tag the managed instance with your id as the owner. You can find the id by running `aws sts get-caller-identity` command on your local machine. The value will be returned as `UserId`.

In [ ]:
!rm -fr ~/.sm-ssh-owner

In [ ]:
%%sh
# LOCAL_USER_ID="AIDACKCEVSQ6C2EXAMPLE"  # replace with your local UserId ( IAM user AK)

if [ -f ~/.sm-ssh-owner ]; then
    echo "sm-ssh-notebook: SSH owner user ID is already set in ~/.sm-ssh-owner, skipping override"
else
    echo "sm-ssh-notebook: Saving SSH owner user ID into ~/.sm-ssh-owner"
    echo "$LOCAL_USER_ID" > ~/.sm-ssh-owner
fi

In [ ]:
with open(os.path.expanduser('~/.sm-ssh-owner')) as f:
    local_user_id = f.readline().rstrip()

Now start the estimator with SSH wrapper. It will connect to the SSM and start SSH server inside the container.

Keep the cell running.

On the local machne, open the SSH tunnel to the notebook with `sm-local-ssh-notebook connect <<<notebook-instance-name>>`.

The estimator will mirror the current notebook directory to `/opt/ml/input/data/notebook` inside the container. You can use it to configure the mappings in your local IDE, so that your local files are synchronized with notebook instance files. See [the instructions for PyCharm](https://www.jetbrains.com/help/pycharm/deployment-mappings-tab.html), as an example.

In [ ]:
estimator = PyTorch(entry_point='ssh_notebook.py',
                    source_dir='ssh_notebook/',
                    dependencies=[SSHEstimatorWrapper.dependency_dir()],
                    base_job_name='ssh-notebook',
                    role=sagemaker.get_execution_role(),
                    framework_version='1.9.1',
                    py_version='py38',
                    instance_count=1,
                    instance_type='local',
                    # instance_type='ml.m5.xlarge',
                    container_log_level=logging.INFO)

ssh_wrapper = SSHEstimatorWrapper.create(estimator, connection_wait_time_seconds=0, local_user_id=local_user_id, log_to_stdout=True)

estimator.fit({'notebook': 'file://'})

In [ ]:
instance_ids = ssh_wrapper.get_instance_ids()  # <--NEW--

logging.info(f"To connect over SSM run: aws ssm start-session --target {instance_ids[0]}")
logging.info(f"To connect over SSH run: sm-local-ssh-training connect {ssh_wrapper.training_job_name()}")

At your local laptop:
```shell
pip install sagemaker-ssh-helper
```

Alternatively, if you want to work on a different instance type, submit the training job to SageMaker cluster as described in [Remote code execution with PyCharm / VSCode over SSH](https://github.com/aws-samples/sagemaker-ssh-helper#remote-interpreter) and use `sm-local-ssh-training` command instead of `sm-local-ssh-notebook`. 


```shell
aws ssm start-session --target mi-xxx --profile ws
```


ps xfaww - Show running tree of processes
ps xfawwe - Show running tree of processes with environment variables
ls -l /opt/ml/input/data - Show input channels
ls -l /opt/ml/code - Show your training code
pip freeze | less - Show all Python packages installed
dpkg -l | less - Show all system packages installed